In [ ]:
pip install tensorflow numpy opencv-python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path to the original dataset directory
original_dataset_dir = '/content/drive/MyDrive/Defense_GNN/Selected Image/Model/Sorted Marge Data'  # Replace with the path to your original dataset

# Paths to the new train, validation, and test directories within the original dataset directory
train_dataset_dir = os.path.join(original_dataset_dir, 'train')
val_dataset_dir = os.path.join(original_dataset_dir, 'validation')
test_dataset_dir = os.path.join(original_dataset_dir, 'test')

# Create the new directories if they don't exist
os.makedirs(train_dataset_dir, exist_ok=True)
os.makedirs(val_dataset_dir, exist_ok=True)
os.makedirs(test_dataset_dir, exist_ok=True)

# Function to split data into train, validation, and test sets
def split_data(original_dir, train_dir, val_dir, test_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    for class_name in os.listdir(original_dir):
        class_dir = os.path.join(original_dir, class_name)
        if os.path.isdir(class_dir) and class_name not in ['train', 'validation', 'test']:
            images = os.listdir(class_dir)
            np.random.shuffle(images)
            train_split = int(len(images) * train_ratio)
            val_split = int(len(images) * (train_ratio + val_ratio))

            train_images = images[:train_split]
            val_images = images[train_split:val_split]
            test_images = images[val_split:]

            # Create class subdirectories
            os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
            os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
            os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

            # Move images to the respective directories
            for image in train_images:
                shutil.copy(os.path.join(class_dir, image), os.path.join(train_dir, class_name, image))
            for image in val_images:
                shutil.copy(os.path.join(class_dir, image), os.path.join(val_dir, class_name, image))
            for image in test_images:
                shutil.copy(os.path.join(class_dir, image), os.path.join(test_dir, class_name, image))

# Split the dataset
split_data(original_dataset_dir, train_dataset_dir, val_dataset_dir, test_dataset_dir)

# Data generators for training, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Number of classes
num_classes = len(train_generator.class_indices)

print(f'Number of classes: {num_classes}')
print('Class indices:', train_generator.class_indices)

Found 2498 images belonging to 7 classes.
Found 988 images belonging to 7 classes.
Found 520 images belonging to 7 classes.
Number of classes: 7
Class indices: {'Bike': 0, 'Couse Bus': 1, 'Micro Bus': 2, 'SUV': 3, 'Sedan Car': 4, 'Taxi': 5, 'Truck or Tanker': 6}


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
def create_cnn_model(input_shape, num_classes):
    model = Sequential()

    # Convolutional Layer 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    # Convolutional Layer 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    # Convolutional Layer 3
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    # Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Input shape (assuming 224x224 RGB images)
input_shape = (224, 224, 3)

# Create the model
model = create_cnn_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    epochs=25,  # You can adjust the number of epochs
    validation_data=val_generator
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('cnn_baseline_model.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('mobilenetv2_finetuned.h5')

9406464/9406464 [==============================] - 2s 0us/step
Epoch 1/10
79/79 [==============================] - 34s 362ms/step - loss: 1.1149 - accuracy: 0.6149 - val_loss: 0.8576 - val_accuracy: 0.7024
Epoch 2/10
79/79 [==============================] - 27s 338ms/step - loss: 0.8049 - accuracy: 0.7102 - val_loss: 0.7635 - val_accuracy: 0.7217
Epoch 3/10
79/79 [==============================] - 27s 342ms/step - loss: 0.7197 - accuracy: 0.7450 - val_loss: 0.7164 - val_accuracy: 0.7429
Epoch 4/10
79/79 [==============================] - 27s 343ms/step - loss: 0.6575 - accuracy: 0.7530 - val_loss: 0.6546 - val_accuracy: 0.7591
Epoch 5/10
79/79 [==============================] - 27s 339ms/step - loss: 0.5964 - accuracy: 0.7814 - val_loss: 0.6067 - val_accuracy: 0.7763
Epoch 6/10
79/79 [==============================] - 27s 338ms/step - loss: 0.5387 - accuracy: 0.8122 - val_loss: 0.5691 - val_accuracy: 0.8077
Epoch 7/10
79/79 [==============================] - 27s 340ms/step - loss: 0.48

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('resnet50_finetuned.h5')

94765736/94765736 [==============================] - 5s 0us/step
Epoch 1/10
79/79 [==============================] - 37s 387ms/step - loss: 1.6056 - accuracy: 0.3931 - val_loss: 1.4917 - val_accuracy: 0.4676
Epoch 2/10
79/79 [==============================] - 28s 355ms/step - loss: 1.4360 - accuracy: 0.4744 - val_loss: 1.3757 - val_accuracy: 0.5071
Epoch 3/10
79/79 [==============================] - 27s 348ms/step - loss: 1.3766 - accuracy: 0.5060 - val_loss: 1.3258 - val_accuracy: 0.5172
Epoch 4/10
79/79 [==============================] - 29s 365ms/step - loss: 1.3237 - accuracy: 0.5164 - val_loss: 1.2795 - val_accuracy: 0.5476
Epoch 5/10
79/79 [==============================] - 28s 350ms/step - loss: 1.2813 - accuracy: 0.5452 - val_loss: 1.2517 - val_accuracy: 0.5749
Epoch 6/10
79/79 [==============================] - 28s 350ms/step - loss: 1.2547 - accuracy: 0.5580 - val_loss: 1.2300 - val_accuracy: 0.5800
Epoch 7/10
79/79 [==============================] - 27s 345ms/step - loss: 1.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load the pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('inceptionv3_finetuned.h5')

87910968/87910968 [==============================] - 5s 0us/step
Epoch 1/10
79/79 [==============================] - 40s 422ms/step - loss: 1.1119 - accuracy: 0.6165 - val_loss: 0.8915 - val_accuracy: 0.6953
Epoch 2/10
79/79 [==============================] - 27s 344ms/step - loss: 0.8874 - accuracy: 0.6894 - val_loss: 0.8470 - val_accuracy: 0.6964
Epoch 3/10
79/79 [==============================] - 27s 344ms/step - loss: 0.8017 - accuracy: 0.7178 - val_loss: 0.7651 - val_accuracy: 0.7318
Epoch 4/10
79/79 [==============================] - 27s 343ms/step - loss: 0.7472 - accuracy: 0.7270 - val_loss: 0.7221 - val_accuracy: 0.7379
Epoch 5/10
79/79 [==============================] - 27s 346ms/step - loss: 0.6985 - accuracy: 0.7442 - val_loss: 0.6783 - val_accuracy: 0.7591
Epoch 6/10
79/79 [==============================] - 28s 349ms/step - loss: 0.6539 - accuracy: 0.7454 - val_loss: 0.6594 - val_accuracy: 0.7551
Epoch 7/10
79/79 [==============================] - 27s 344ms/step - loss: 0.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('vgg16_finetuned.h5')

58889256/58889256 [==============================] - 4s 0us/step
Epoch 1/10
79/79 [==============================] - 40s 426ms/step - loss: 1.3756 - accuracy: 0.5084 - val_loss: 1.1696 - val_accuracy: 0.6073
Epoch 2/10
79/79 [==============================] - 27s 337ms/step - loss: 1.1042 - accuracy: 0.6365 - val_loss: 1.0454 - val_accuracy: 0.6478
Epoch 3/10
79/79 [==============================] - 27s 341ms/step - loss: 1.0177 - accuracy: 0.6609 - val_loss: 0.9802 - val_accuracy: 0.6741
Epoch 4/10
79/79 [==============================] - 30s 386ms/step - loss: 0.9672 - accuracy: 0.6681 - val_loss: 0.9354 - val_accuracy: 0.6751
Epoch 5/10
79/79 [==============================] - 27s 346ms/step - loss: 0.9272 - accuracy: 0.6857 - val_loss: 0.9037 - val_accuracy: 0.6974
Epoch 6/10
79/79 [==============================] - 27s 340ms/step - loss: 0.9024 - accuracy: 0.6930 - val_loss: 0.8821 - val_accuracy: 0.6964
Epoch 7/10
79/79 [==============================] - 27s 344ms/step - loss: 0.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load the pre-trained EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('efficientnetb0_finetuned.h5')

16705208/16705208 [==============================] - 2s 0us/step
Epoch 1/10
79/79 [==============================] - 39s 373ms/step - loss: 1.6435 - accuracy: 0.3679 - val_loss: 1.6233 - val_accuracy: 0.3978
Epoch 2/10
79/79 [==============================] - 26s 332ms/step - loss: 1.6288 - accuracy: 0.3883 - val_loss: 1.6252 - val_accuracy: 0.2540
Epoch 3/10
79/79 [==============================] - 26s 332ms/step - loss: 1.6226 - accuracy: 0.3843 - val_loss: 1.6150 - val_accuracy: 0.3978
Epoch 4/10
79/79 [==============================] - 26s 331ms/step - loss: 1.6143 - accuracy: 0.4043 - val_loss: 1.6021 - val_accuracy: 0.3978
Epoch 5/10
79/79 [==============================] - 26s 327ms/step - loss: 1.6094 - accuracy: 0.4027 - val_loss: 1.6082 - val_accuracy: 0.3978
Epoch 6/10
79/79 [==============================] - 26s 330ms/step - loss: 1.6130 - accuracy: 0.3955 - val_loss: 1.5987 - val_accuracy: 0.3978
Epoch 7/10
79/79 [==============================] - 26s 329ms/step - loss: 1.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load the pre-trained DenseNet201 model
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('densenet201_finetuned.h5')

74836368/74836368 [==============================] - 4s 0us/step
Epoch 1/10
79/79 [==============================] - 64s 592ms/step - loss: 1.1091 - accuracy: 0.6309 - val_loss: 0.8591 - val_accuracy: 0.7045
Epoch 2/10
79/79 [==============================] - 28s 358ms/step - loss: 0.8466 - accuracy: 0.7026 - val_loss: 0.7855 - val_accuracy: 0.6953
Epoch 3/10
79/79 [==============================] - 28s 358ms/step - loss: 0.7728 - accuracy: 0.7202 - val_loss: 0.7166 - val_accuracy: 0.7348
Epoch 4/10
79/79 [==============================] - 29s 370ms/step - loss: 0.7050 - accuracy: 0.7378 - val_loss: 0.6983 - val_accuracy: 0.7298
Epoch 5/10
79/79 [==============================] - 28s 353ms/step - loss: 0.6892 - accuracy: 0.7350 - val_loss: 0.7293 - val_accuracy: 0.7156
Epoch 6/10
79/79 [==============================] - 28s 357ms/step - loss: 0.6423 - accuracy: 0.7562 - val_loss: 0.6504 - val_accuracy: 0.7581
Epoch 7/10
79/79 [==============================] - 28s 358ms/step - loss: 0.